<a href="https://colab.research.google.com/github/nightly228/marketplace-LLM/blob/main/%D0%BB%D0%BB%D0%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import pandas as pd

# Загружаем файлы с компьютера
uploaded = files.upload()  # выбери train.csv, test.csv, submission_example.csv

# Теперь можно читать их просто по имени
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample = pd.read_csv("submission_example.csv")

print(train.head())
print(test.head())
print(sample.head())


Saving submission_example.csv to submission_example.csv
Saving test.csv to test.csv
Saving train.csv to train.csv
                                                text
0  Заказали 14.10.2017 , получили 25.10.2017 \r\n...
1  футболка хорошего качества,но футболка не как ...
2                                     Все отлично!!!
3  Рисунок не очень чёткий, а ткань прозрачная, в...
4  плохо!!!Низ рваный..деньги не вернули!Открыла ...
                                                text
0                                   Советую продавца
1  По вашему это платье???? Это узкая кофта !!!! ...
2  Жуткая синтетика. Неприятная ткань. Летом не п...
3  Джемперок так себе на хилую четверку,запах гол...
4  обычная х/б рубашка.не плотная,просвечивает ни...
          category
0  бытовая техника
1       нет товара
2            обувь
3           одежда
4           посуда


In [2]:
!pip install -q sentence-transformers

import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample = pd.read_csv("submission_example.csv")


categories = sample['category'].unique().tolist()
print("Категории:", categories)


model = SentenceTransformer("cointegrated/rubert-tiny")


category_embeddings = model.encode(categories, convert_to_tensor=True, show_progress_bar=True)

def get_category(text):
    text_emb = model.encode(text, convert_to_tensor=True)
    cos_scores = util.cos_sim(text_emb, category_embeddings)[0]
    best_idx = torch.argmax(cos_scores).item()
    return categories[best_idx]



# train.to_csv("train_pseudo.csv", index=False)
# print(train.head())

Категории: ['бытовая техника', 'нет товара', 'обувь', 'одежда', 'посуда', 'текстиль', 'товары для детей', 'украшения и аксессуары', 'электроника']


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.7M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
train["pseudo_category"] = train["text"].apply(get_category)
display(train.head())

,text,pseudo_category
0,"Заказали 14.10.2017 , получили 25.10.2017 \r\n...",обувь
1,"футболка хорошего качества,но футболка не как ...",бытовая техника
2,Все отлично!!!,одежда
3,"Рисунок не очень чёткий, а ткань прозрачная, в...",обувь
4,плохо!!!Низ рваный..деньги не вернули!Открыла ...,обувь


In [4]:
train.to_csv("train_pseudo.csv", index=False)
print(train.head())

                                                text  pseudo_category
0  Заказали 14.10.2017 , получили 25.10.2017 \r\n...            обувь
1  футболка хорошего качества,но футболка не как ...  бытовая техника
2                                     Все отлично!!!           одежда
3  Рисунок не очень чёткий, а ткань прозрачная, в...            обувь
4  плохо!!!Низ рваный..деньги не вернули!Открыла ...            обувь


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

X = train["text"]
y = train["pseudo_category"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

vec = TfidfVectorizer(max_features=50000, ngram_range=(1,2))
X_train_vec = vec.fit_transform(X_train)
X_val_vec = vec.transform(X_val)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_vec, y_train)

y_pred = clf.predict(X_val_vec)
print("Weighted F1 на валидации:", f1_score(y_val, y_pred, average="weighted"))


Weighted F1 на валидации: 0.45679987949016687


In [8]:
X_test_vec = vec.transform(test["text"])
preds_test = clf.predict(X_test_vec)

submission = pd.DataFrame({
    "id": range(len(test)),
    "category": preds_test
})
submission.to_csv("submission.csv", index=False)
print(submission.head())

   id category
0   0   посуда
1   1    обувь
2   2    обувь
3   3    обувь
4   4    обувь


In [7]:
train["pseudo_category"].value_counts()


,count
pseudo_category,
обувь,708
нет товара,406
посуда,305
украшения и аксессуары,204
бытовая техника,92
одежда,48
товары для детей,27
текстиль,25
электроника,3
